<a href="https://colab.research.google.com/github/Nmg1994/Crop_mapping/blob/main/Crop_mapping_Region_based_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Intalling necessary packages

In [ ]:
!pip install rasterio
!pip install tqdm

# Importing necessary libraries/modules

In [2]:
import numpy as np
import seaborn as sns
import matplotlib.pylab as plt
from sklearn.preprocessing import StandardScaler
import sklearn.metrics
from sklearn.preprocessing import OneHotEncoder
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Flatten, Concatenate, Layer, Conv1D, MaxPooling1D
from keras.callbacks import EarlyStopping
import pickle
from scipy.stats import ks_2samp
from itertools import combinations

# Mounting Google Drive

In [3]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


# Loading training, validation, and test datasets as well as the class weight

In [4]:
# loading samples along with their corresponding labels # this is done to save some RAM due to the huge size of data
train_indices = np.load('/content/drive/My Drive/DL_samples/train_indices.npy')
val_indices = np.load('/content/drive/My Drive/DL_samples/val_indices.npy')
test_indices = np.load('/content/drive/My Drive/DL_samples/test_indices.npy')

train_labels = np.load('/content/drive/My Drive/DL_samples/train_labels.npy')
val_labels = np.load('/content/drive/My Drive/DL_samples/val_labels.npy')
test_labels = np.load('/content/drive/My Drive/DL_samples/test_labels.npy')


train_samples = np.load('/content/drive/My Drive/DL_samples/train_samples.npy')
val_samples = np.load('/content/drive/My Drive/DL_samples/val_samples.npy')
test_samples = np.load('/content/drive/My Drive/DL_samples/test_samples.npy')

# Loading the class weight dictionary from the Google Drive
with open('/content/drive/My Drive/DL_samples/class_weights_dict.pkl', 'rb') as f:
  class_weights_dict = pickle.load(f)

# Assessing whether the datasets conform the IID assumption

In [ ]:
# Investigating the conformity of the datasets to the IID assumption
# Calculating class distributions for each dataset
train_class_distribution = np.bincount(train_labels) / len(train_labels)
val_class_distribution = np.bincount(val_labels) / len(val_labels)
test_class_distribution = np.bincount(test_labels) / len(test_labels)

# Printing class distributions for each dataset
print("Training set class distribution:", train_class_distribution)
print("Validation set class distribution:", val_class_distribution)
print("Testing set class distribution:", test_class_distribution)

# Defining a function to perform Kolmogorov-Smirnov test and Chi-squared test
def perform_tests(labels1, labels2, dataset_name1, dataset_name2):
    # Performing Kolmogorov-Smirnov test
    ks_statistic, p_value_ks = ks_2samp(labels1, labels2)
    print(f"Kolmogorov-Smirnov test statistic for {dataset_name1} vs. {dataset_name2} datasets:", ks_statistic)
    print(f"p-value for Kolmogorov-Smirnov test between {dataset_name1} vs. {dataset_name2} datasets:", p_value_ks)

# Performing pairwise comparisons between all three datasets
datasets = [("Training", train_labels), ("Validation", val_labels), ("Test", test_labels)]
for dataset1, dataset2 in combinations(datasets, 2):
    dataset_name1, labels1 = dataset1
    dataset_name2, labels2 = dataset2
    perform_tests(labels1, labels2, dataset_name1, dataset_name2)

# Preprocessing the datasets and their corresponding labels

In [ ]:
# One-hot encoding labels
labels = np.concatenate([train_labels, val_labels, test_labels])
print('labels shape:', labels.shape)
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
one_hot_encoded = encoder.fit_transform(labels.reshape(-1, 1))
one_hot_array = np.array(one_hot_encoded)
print('one_hot_array shape:', one_hot_array.shape)

train_labels_encoded = one_hot_array[:len(train_labels),:]
val_labels_encoded = one_hot_array[len(train_labels): (len(train_labels) + len(val_labels)),:]
test_labels_encoded = one_hot_array[(len(train_labels) + len(val_labels)):,:]

print('train_labels_encoded shape:', train_labels_encoded.shape)
print('val_labels_encoded shape:', val_labels_encoded.shape)
print('test_labels_encoded shape:', test_labels_encoded.shape)

In [8]:
# Defining function for the calculation of indices
def indices_calculation(DL_samples_array):

  # Extracting individual bands
  S2_Blue = DL_samples_array[:,0,:]  # Blue band
  S2_Green = DL_samples_array[:,1,:]  # Green band
  S2_Red = DL_samples_array[:,2,:]  # Red band
  S2_NIR = DL_samples_array[:,3,:]  # NIR band

  # Calculation of vegetation indices (VI), NDWI, and BI
  # NDVI
  NDVI = ((S2_NIR - S2_Red) / (S2_NIR + S2_Red))
  NDVI_reshaped = np.expand_dims(NDVI, axis=1)  # Adding a new axis to make it 3D
  DL_samples_array = np.concatenate([DL_samples_array,NDVI_reshaped], axis = 1)

  # GNDVI
  GNDVI = ((S2_NIR - S2_Green) / (S2_NIR + S2_Green))
  GNDVI_reshaped = np.expand_dims(GNDVI, axis=1)  # Adding a new axis to make it 3D
  DL_samples_array = np.concatenate([DL_samples_array,GNDVI_reshaped], axis = 1)

  # GSAVI
  GSAVI = ((S2_NIR - S2_Green) / (S2_NIR + S2_Green + 0.5)) * 1.5
  GSAVI_reshaped = np.expand_dims(GSAVI, axis=1)  # Adding a new axis to make it 3D
  DL_samples_array = np.concatenate([DL_samples_array,GSAVI_reshaped], axis = 1)

  # GOSAVI
  GOSAVI = ((S2_NIR - S2_Green) / (S2_NIR + S2_Green+ 0.16))
  GOSAVI_reshaped = np.expand_dims(GOSAVI, axis=1)  # Adding a new axis to make it 3D
  DL_samples_array = np.concatenate([DL_samples_array,GOSAVI_reshaped], axis = 1)

  # SAVI
  SAVI = ((S2_NIR - S2_Red) / (S2_NIR + S2_Red + 0.5)) * 1.5
  SAVI_reshaped = np.expand_dims(SAVI, axis=1)  # Adding a new axis to make it 3D
  DL_samples_array = np.concatenate([DL_samples_array,SAVI_reshaped], axis = 1)

  # OSAVI
  OSAVI = ((S2_NIR - S2_Red) / (S2_NIR + S2_Red + 0.16)) * 1.16
  OSAVI_reshaped = np.expand_dims(OSAVI, axis=1)  # Adding a new axis to make it 3D
  DL_samples_array = np.concatenate([DL_samples_array,OSAVI_reshaped], axis = 1)

  # NDWI
  NDWI = ((S2_Green - S2_NIR) / (S2_Green + S2_NIR))
  NDWI_reshaped = np.expand_dims(NDWI, axis=1)  # Adding a new axis to make it 3D
  DL_samples_array = np.concatenate([DL_samples_array,NDWI_reshaped], axis = 1)

  # BI
  BI = np.sqrt((S2_Blue ** 2) + (S2_Green ** 2) + (S2_NIR ** 2) + (S2_Red ** 2))
  BI_reshaped = np.expand_dims(BI, axis=1)  # Adding a new axis to make it 3D
  DL_samples_array = np.concatenate([DL_samples_array,BI_reshaped], axis = 1)

  # EVI
  EVI = np.where((S2_NIR + 6 * S2_Red - 7.5 * S2_Blue + 1) != 0, 2.5 * ((S2_NIR - S2_Red) / (S2_NIR + 6 * S2_Red - 7.5 * S2_Blue + 1)), 0)
  EVI_reshaped = np.expand_dims(EVI, axis=1)  # Adding a new axis to make it 3D
  DL_samples_array = np.concatenate([DL_samples_array,EVI_reshaped], axis = 1)

  return DL_samples_array

# Standardizing each band along with the indices calculated
def standarized_function(DL_samples):
  for j in range(DL_samples.shape[1]):
    scaler = StandardScaler()
    each_band_standarized = scaler.fit_transform(DL_samples[:,j,:])
    each_band_standarized_reshaped = np.expand_dims(each_band_standarized, axis=1)

    if j == 0:
      Standarizad_samples = each_band_standarized_reshaped
    else:
      Standarizad_samples= np.concatenate([Standarizad_samples, each_band_standarized_reshaped], axis = 1)
  print('Standarizad_samples shape is: ', Standarizad_samples.shape)
  return Standarizad_samples

In [ ]:
# Applying indices_calculation function to calculate the desired indices
# This cell only must be run in the cases whose objective is to include VIs, NDWI, and BI in the process
train_samples_all_indices = indices_calculation(train_samples)
val_samples_all_indices = indices_calculation(val_samples)
test_samples_all_indices = indices_calculation(test_samples)

# Standardizing the indices calculated
Standarizad_train_samples = standarized_function(train_samples_all_indices)
Standarizad_val_samples = standarized_function(val_samples_all_indices)
Standarizad_test_samples = standarized_function(test_samples_all_indices)

In [10]:
# Considering EVI as the only spectral feature
# This cell only must be run in the cases whose objective is to consider the EVI as the only spectral feature in the models
train_samples_all_indices = indices_calculation(train_samples)
val_samples_all_indices = indices_calculation(val_samples)
test_samples_all_indices = indices_calculation(test_samples)

# Standardizing the indices calculated
Standarizad_train_samples = standarized_function(train_samples_all_indices)
Standarizad_val_samples = standarized_function(val_samples_all_indices)
Standarizad_test_samples = standarized_function(test_samples_all_indices)

# Considering EVI as the only spectral feature
Standarizad_train_samples = Standarizad_train_samples[:,-1,:]
Standarizad_val_samples = Standarizad_val_samples[:,-1,:]
Standarizad_test_samples = Standarizad_test_samples[:,-1,:]

# Reshaping the arrays in a suitable shape for being fed in the models
Standarizad_train_samples = Standarizad_train_samples.reshape(Standarizad_train_samples.shape[0], 1, Standarizad_train_samples.shape[1])
Standarizad_val_samples = Standarizad_val_samples.reshape(Standarizad_val_samples.shape[0], 1, Standarizad_val_samples.shape[1])
Standarizad_test_samples = Standarizad_test_samples.reshape(Standarizad_test_samples.shape[0], 1, Standarizad_test_samples.shape[1])

In [11]:
# Transposing the spectral and temporal domains to apply the convolutions on the spectral domain
# This cell must be run only in the cases whose objective is to apply convolutions over the spectral domain (i.e., using 1DSpecCNNs model)
Standarizad_train_samples= np.transpose(Standarizad_train_samples, (0,2,1))
Standarizad_val_samples= np.transpose(Standarizad_val_samples, (0,2,1))
Standarizad_test_samples= np.transpose(Standarizad_test_samples, (0,2,1))

# Modifying the initial architecures of each DL model and exploring the one with optimal performance and using them for the prediction and accuracy assessment on the testing dataset

Defininf the LSTM model with the optimal architecture

In [ ]:
# LSTM model
def LSTM_classification(Features_training_LSTM, Features_val_LSTM, Features_testing_LSTM, landcover_training_LSTM, landcover_val_LSTM, landcover_testing_LSTM, time_steps, num_features, num_classes, Prediction_only = False):

  # Defining early stopping callback
  early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

  # Designing the architecture for the LSTM model
  LSTM_model = Sequential()
  LSTM_model.add(LSTM(units = 256, return_sequences= True, input_shape=(num_features, time_steps)))
  LSTM_model.add(Dropout(0.1))
  LSTM_model.add(LSTM(units = 128, return_sequences= True))
  LSTM_model.add(Dropout(0.1))
  LSTM_model.add(Flatten())
  LSTM_model.add(Dense(units = 512, activation = 'relu'))
  LSTM_model.add(Dense(num_classes, activation='softmax'))

  learning_rate = 0.001
  adam_optimizer = keras.optimizers.Adam(learning_rate=learning_rate)

  # Compiling the LSTM model
  LSTM_model.compile(optimizer= adam_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

  # Training the LSTM model
  LSTM_history = LSTM_model.fit(Features_training_LSTM, landcover_training_LSTM, epochs=100, batch_size=32, validation_data=(Features_val_LSTM, landcover_val_LSTM),  callbacks=[early_stopping])

  # Access the stopped epoch
  final_epoch = early_stopping.stopped_epoch
  print("Training stopped at epoch:", final_epoch + 1)

  sns.set(font_scale=1)
  sns.set_style("darkgrid")
  plt.figure(figsize=(15, 4))

  print('The average accuracy over all epochs: ', sum(LSTM_history.history['val_accuracy']) / len(LSTM_history.history['val_accuracy']))

  # Plotting training accuracy
  sns.lineplot(x=range(1, final_epoch + 2), y= LSTM_history.history['accuracy'], label='Training accuracy')

  # Plotting validation accuracy
  sns.lineplot(x=range(1, final_epoch + 2), y= LSTM_history.history['val_accuracy'], label='Validation accuracy')

  plt.xlabel('Epoch')
  plt.ylabel('Accuracy')
  plt.title('Training and Validation accuracy of LSTM model Over Epochs')
  plt.legend()
  plt.show()

  LSTM_model.summary()


  #Testing Accuracy
  landcover_testing_predicted = LSTM_model.predict(Features_testing_LSTM)
  # Finding the index of the highest probability for each sample
  max_prob_indices = np.argmax(landcover_testing_predicted, axis=1)

  # Creating a binary array with the same shape as landcover_testing_predicted
  binary_array = np.zeros_like(landcover_testing_predicted)

  # Setting the value of 1 at the index of the highest probability for each sample
  binary_array[np.arange(len(landcover_testing_predicted)), max_prob_indices] = 1


  if Prediction_only == False:
    print("Recall score for test dataset: ", sklearn.metrics.recall_score(landcover_testing_LSTM, binary_array, average = 'macro'))
    print("Precision score for test dataset: ", sklearn.metrics.precision_score(landcover_testing_LSTM, binary_array, average = 'macro'))
    print("F1 score for test dataset: ", sklearn.metrics.f1_score(landcover_testing_LSTM, binary_array, average = 'macro'))
    print("ROC-AUC score for test dataset: ", sklearn.metrics.roc_auc_score(landcover_testing_LSTM, binary_array, average = 'macro', multi_class = 'ovr'))

    print(sklearn.metrics.classification_report(landcover_testing_LSTM, binary_array))

    # Creating an empty dictionary to store ROC AUC scores for each class
    roc_auc_scores = {}

    # Iterating over each class
    for class_index in range(num_classes):
        # Extracting the binary predictions for the current class
        binary_predictions_class = binary_array[:, class_index]
        true_labels_class = landcover_testing_LSTM[:, class_index]

        # Computing the ROC-AUC score for the current class
        roc_auc = sklearn.metrics.roc_auc_score(true_labels_class, binary_predictions_class)

        # Storing the ROC AUC score for the current class in the dictionary
        roc_auc_scores[f'Class {class_index}'] = roc_auc

    # Printing ROC-AUC scores for each class
    for class_name, roc_auc in roc_auc_scores.items():
        print(f'ROC AUC for {class_name}: {roc_auc}')

Applying LSTM model

In [ ]:
# Following the goal set the inputs of the LSTM model:
# If only four bands are being used, use the inputs train_samples, val_samples, test_samples; other wise use the inputs Standarizad_train_samples, Standarizad_val_samples, Standarizad_test_samples
# Also, do not forget to correspondingly change the time_steps and num_features parameters
LSTM_classification(Features_training_LSTM = train_samples, Features_val_LSTM = val_samples, Features_testing_LSTM = test_samples, landcover_training_LSTM = train_labels_encoded, landcover_val_LSTM = val_labels_encoded, landcover_testing_LSTM = test_labels_encoded, time_steps = train_samples.shape[2], num_features = train_samples.shape[1], num_classes = 13, Prediction_only = False)


Defininf the inception layer for the CNNs models

In [12]:
# Designing an inception layer for the CNNs models
class InceptionLayer(Layer):
    def __init__(self, filters, **kwargs):
        super(InceptionLayer, self).__init__(**kwargs)
        self.filters = filters
        # Define layers here to avoid tf.function issues
        self.conv1_3 = Conv1D(filters[0], 3, padding='same', activation='relu')
        self.conv1_1 = Conv1D(filters[1], 1, padding='same', activation='relu')

        self.conv2_1 = Conv1D(filters[2], 1, padding='same', activation='relu')
        self.conv2_3 = Conv1D(filters[3], 3, padding='same', activation='relu')

        self.conv3_3 = Conv1D(filters[4], 3, padding='same', activation='relu')
        self.conv3_5 = Conv1D(filters[5], 5, padding='same', activation='relu')

        self.conv4_5 = Conv1D(filters[6], 5, padding='same', activation='relu')
        self.conv4_3 = Conv1D(filters[7], 3, padding='same', activation='relu')

        self.conv5_5 = Conv1D(filters[8], 5, padding='same', activation='relu')
        self.conv5_1 = Conv1D(filters[9], 1, padding='same', activation='relu')

        self.conv6_1 = Conv1D(filters[10], 1, padding='same', activation='relu')
        self.conv6_5 = Conv1D(filters[11], 5, padding='same', activation='relu')

        self.pool = MaxPooling1D(pool_size=2, strides=1, padding='same')
        self.conv_pool = Conv1D(filters[12], 3, padding='same', activation='relu')

    def call(self, inputs):
        # Branch 1: 3 Convolution followed by 1 Convolution
        branch1 = self.conv1_3(inputs)
        branch1 = self.conv1_1(branch1)

        # Branch 2: 1 Convolution followed by 3 Convolution
        branch2 = self.conv2_1(inputs)
        branch2 = self.conv2_3(branch2)

        # Branch 3: 3 Convolution followed by 5 Convolution
        branch3 = self.conv3_3(inputs)
        branch3 = self.conv3_5(branch3)

        # Branch 4: 5 Convolution followed by 3 Convolution
        branch4 = self.conv4_5(inputs)
        branch4 = self.conv4_3(branch4)

        # Branch 5: 5 Convolution followed by 1 Convolution
        branch5 = self.conv5_5(inputs)
        branch5 = self.conv5_1(branch5)

        # Branch 6: 1 Convolution followed by 5 Convolution
        branch6 = self.conv6_1(inputs)
        branch6 = self.conv6_5(branch6)

        # Branch pool: Max pooling followed by 3 Convolution
        branch_pool = self.pool(inputs)
        branch_pool = self.conv_pool(branch_pool)

        # Concatenate the outputs of the branches
        output = Concatenate(axis=-1)([branch1, branch4, branch5])

        return output

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[1], sum(self.filters)


Defining the 1DTempCNNs model with the optimal architecture

In [ ]:
# 1DTempCNNs model
def TempCNNs(Features_training_CNN, Features_val_CNN, Features_testing_CNN, landcover_training_CNN, landcover_val_CNN, landcover_testing_CNN, time_steps, num_features, num_classes, Prediction_only = False):

    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    # Designing the architecture for the 1DTempCNNs model
    CNN_model = Sequential()

    CNN_model.add(Conv1D(128, kernel_size=3, activation='relu', padding='same', input_shape=(num_features, time_steps)))
    CNN_model.add(Dropout(0.1))
    CNN_model.add(InceptionLayer((64, 64, 128, 128, 256, 256, 128, 128, 64, 64, 256, 128, 128)))
    CNN_model.add(Dropout(0.1))
    CNN_model.add(Conv1D(128, kernel_size=3, activation='relu', padding='same'))
    CNN_model.add(Dropout(0.1))
    CNN_model.add(Conv1D(128, kernel_size=3, activation='relu', padding='same'))
    CNN_model.add(Dropout(0.1))
    # Flatten layer
    CNN_model.add(Flatten())

    # Fully connected layers
    CNN_model.add(Dense(512, activation='relu'))
    # Output layer
    CNN_model.add(Dense(num_classes, activation='softmax'))

    # Compiling the 1DTempCNNs model
    CNN_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # Training the 1DTempCNNs model
    CNN_history = CNN_model.fit(Features_training_CNN, landcover_training_CNN, epochs=100, batch_size=32, validation_data=(Features_val_CNN, landcover_val_CNN),  callbacks=[early_stopping])

    # Accessing the stopped epoch
    final_epoch = early_stopping.stopped_epoch
    print("Training stopped at epoch:", final_epoch + 1)

    sns.set(font_scale=1)
    sns.set_style("darkgrid")
    plt.figure(figsize=(15, 4))

    print('The average accuracy over all epochs: ', sum(CNN_history.history['val_accuracy']) / len(CNN_history.history['val_accuracy']))

    # Plotting training accuracy
    sns.lineplot(x=range(1, final_epoch + 2), y= CNN_history.history['accuracy'], label='Training accuracy')

    # Plotting validation accuracy
    sns.lineplot(x=range(1, final_epoch + 2), y= CNN_history.history['val_accuracy'], label='Validation accuracy')

    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Training and Validation accuracy of 1DTempCNNs Over Epochs')
    plt.legend()
    plt.show()

    CNN_model.summary()

    #Testing Accuracy
    landcover_testing_predicted = CNN_model.predict(Features_testing_CNN)
    # Finding the index of the highest probability for each sample
    max_prob_indices = np.argmax(landcover_testing_predicted, axis=1)

    # Creating a binary array with the same shape as landcover_testing_predicted
    binary_array = np.zeros_like(landcover_testing_predicted)

    # Setting the value of 1 at the index of the highest probability for each sample
    binary_array[np.arange(len(landcover_testing_predicted)), max_prob_indices] = 1

    if Prediction_only == False:
      print("Recall score for test dataset: ", sklearn.metrics.recall_score(landcover_testing_CNN, binary_array, average = 'macro'))
      print("Precision score for test dataset: ", sklearn.metrics.precision_score(landcover_testing_CNN, binary_array, average = 'macro'))
      print("F1 score for test dataset: ", sklearn.metrics.f1_score(landcover_testing_CNN, binary_array, average = 'macro'))
      print("ROC-AUC score for test dataset: ", sklearn.metrics.roc_auc_score(landcover_testing_CNN, binary_array, average = 'macro', multi_class = 'ovr'))

      print(sklearn.metrics.classification_report(landcover_testing_CNN, binary_array))

      # Creating an empty dictionary to store ROC-AUC scores for each class
      roc_auc_scores = {}

      # Iterating over each class
      for class_index in range(num_classes):
          # Extracting the binary predictions for the current class
          binary_predictions_class = binary_array[:, class_index]
          true_labels_class = landcover_testing_CNN[:, class_index]

          # Computing ROC-AUC score for the current class
          roc_auc = sklearn.metrics.roc_auc_score(true_labels_class, binary_predictions_class)

          # Storing the ROC-AUC score for the current class in the dictionary
          roc_auc_scores[f'Class {class_index}'] = roc_auc

      # Printing ROC-AUC scores for each class
      for class_name, roc_auc in roc_auc_scores.items():
          print(f'ROC AUC for {class_name}: {roc_auc}')


Applying the 1DTempCNNs model

In [ ]:
# Following the goal set the inputs of the 1DTempCNNs model:
# If only four bands are being used, use the inputs train_samples, val_samples, test_samples; other wise use the inputs Standarizad_train_samples, Standarizad_val_samples, Standarizad_test_samples
# Also, do not forget to correspondingly change the time_steps and num_features parameters
TempCNNs(Features_training_CNN = train_samples, Features_val_CNN = val_samples, Features_testing_CNN = test_samples, landcover_training_CNN = train_labels_encoded, landcover_val_CNN = val_labels_encoded, landcover_testing_CNN = test_labels_encoded, time_steps = train_samples.shape[2], num_features = train_samples.shape[1], num_classes = 13, Prediction_only = False)


Defining the 1DSpecCNNs model with the optimal architecture

In [ ]:
# 1DSpecCNNs model
def SpecCNNs(Features_training_CNN, Features_val_CNN, Features_testing_CNN, landcover_training_CNN, landcover_val_CNN, landcover_testing_CNN, time_steps, num_features, num_classes, Prediction_only = False):

    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    # Designing the architecture for the 1DSpecCNNs model
    CNN_model = Sequential()

    CNN_model.add(Conv1D(64, kernel_size=3, activation='relu', padding='same', input_shape=(time_steps, num_features)))
    CNN_model.add(Dropout(0.1))
    CNN_model.add(InceptionLayer((32, 32, 128, 128, 256, 256, 64, 64, 32, 32, 256, 128, 128)))
    CNN_model.add(Dropout(0.1))
    # Flatten layer
    CNN_model.add(Flatten())

    # Fully connected layers
    CNN_model.add(Dense(512, activation='relu'))
    # Output layer
    CNN_model.add(Dense(num_classes, activation='softmax'))

    # Compiling the 1DSpecCNNs model
    CNN_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    CNN_model.summary()

    # Training the 1DSpecCNNs model
    CNN_history = CNN_model.fit(Features_training_CNN, landcover_training_CNN, epochs=100, batch_size=32, validation_data=(Features_val_CNN, landcover_val_CNN),  callbacks=[early_stopping])

    # Accessing the stopped epoch
    final_epoch = early_stopping.stopped_epoch
    print("Training stopped at epoch:", final_epoch + 1)

    sns.set(font_scale=1)
    sns.set_style("darkgrid")
    plt.figure(figsize=(15, 4))

    print('The average accuracy over all epochs: ', sum(CNN_history.history['val_accuracy']) / len(CNN_history.history['val_accuracy']))

    # Plotting training accuracy
    sns.lineplot(x=range(1, final_epoch + 2), y= CNN_history.history['accuracy'], label='Training accuracy')

    # Plotting validation accuracy
    sns.lineplot(x=range(1, final_epoch + 2), y= CNN_history.history['val_accuracy'], label='Validation accuracy')

    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Training and Validation accuracy of 1DTempCNNs Over Epochs')
    plt.legend()
    plt.show()

    CNN_model.summary()

    #Testing Accuracy
    landcover_testing_predicted = CNN_model.predict(Features_testing_CNN)
    # Finding the index of the highest probability for each sample
    max_prob_indices = np.argmax(landcover_testing_predicted, axis=1)

    # Creating a binary array with the same shape as landcover_testing_predicted
    binary_array = np.zeros_like(landcover_testing_predicted)

    # Setting the value of 1 at the index of the highest probability for each sample
    binary_array[np.arange(len(landcover_testing_predicted)), max_prob_indices] = 1

    if Prediction_only == False:
      print("Recall score for test dataset: ", sklearn.metrics.recall_score(landcover_testing_CNN, binary_array, average = 'macro'))
      print("Precision score for test dataset: ", sklearn.metrics.precision_score(landcover_testing_CNN, binary_array, average = 'macro'))
      print("F1 score for test dataset: ", sklearn.metrics.f1_score(landcover_testing_CNN, binary_array, average = 'macro'))
      print("ROC-AUC score for test dataset: ", sklearn.metrics.roc_auc_score(landcover_testing_CNN, binary_array, average = 'macro', multi_class = 'ovr'))

      print(sklearn.metrics.classification_report(landcover_testing_CNN, binary_array))

      # Creating an empty dictionary to store ROC-AUC scores for each class
      roc_auc_scores = {}

      # Iterating over each class
      for class_index in range(num_classes):
          # Extracting the binary predictions for the current class
          binary_predictions_class = binary_array[:, class_index]
          true_labels_class = landcover_testing_CNN[:, class_index]

          # Computing ROC-AUC score for the current class
          roc_auc = sklearn.metrics.roc_auc_score(true_labels_class, binary_predictions_class)

          # Storing the ROC-AUC score for the current class in the dictionary
          roc_auc_scores[f'Class {class_index}'] = roc_auc

      # Printing ROC-AUC scores for each class
      for class_name, roc_auc in roc_auc_scores.items():
          print(f'ROC AUC for {class_name}: {roc_auc}')


Applying the 1DSpecCNNs model

In [ ]:
# Following the goal set the inputs of the 1DSpecCNNs model:
# If only four bands are being used, use the inputs train_samples, val_samples, test_samples; other wise use the inputs Standarizad_train_samples, Standarizad_val_samples, Standarizad_test_samples
# Also, do not forget to correspondingly change the time_steps and num_features parameters
SpecCNNs(Features_training_CNN = train_samples, Features_val_CNN = val_samples, Features_testing_CNN = test_samples, landcover_training_CNN = train_labels_encoded, landcover_val_CNN = val_labels_encoded, landcover_testing_CNN = test_labels_encoded, time_steps = train_samples.shape[1], num_features = train_samples.shape[2], num_classes = 13, Prediction_only = False)

